In [4]:
import os
import sys
sys.path.append('..')
import pandas as pd

from scripts.extraccion_datos import *

# Crear la carpeta 'data' si no existe
directorio_salida = 'data'
os.makedirs(directorio_salida, exist_ok=True)

# Descargar el archivo Excel
url = 'https://www.ine.es/jaxiT3/files/t/es/xlsx/60272.xlsx?nocab=1'
ruta_salida = os.path.join(directorio_salida, 'datos_ine.xlsx')
descargar_archivo_excel(url, ruta_salida)

# Verificar que la descarga fue exitosa
if os.path.exists(ruta_salida):
    print("Descarga exitosa del archivo Excel.")
else:
    print("Error en la descarga del archivo Excel.")

# Cargar el archivo Excel
datos_excel = cargar_archivo_excel(ruta_salida)

# Leer la hoja de datos
nombre_hoja = datos_excel.sheet_names[0]
df = leer_hoja_excel(datos_excel, nombre_hoja)

# Guardar el DataFrame original con codificación UTF-8
ruta_salida_original = os.path.join(directorio_salida, 'datos_ine_original.xlsx')
df.to_excel(ruta_salida_original, index=False)

# Mostrar las primeras filas y columnas relevantes del DataFrame
print("Primeras filas del DataFrame original:")
print(df.iloc[:5, :10])  # Mostrar las primeras 5 filas y 10 columnas

Descarga exitosa del archivo Excel.


/usr/local/python/3.12.1/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


Primeras filas del DataFrame original:
0       Índice de Producción Industrial. (Base 2021)     NaN  NaN  NaN  NaN  \
0                                                NaN     NaN  NaN  NaN  NaN   
1  General y por destino económico de los bienes....     NaN  NaN  NaN  NaN   
2                        Unidades:   Índice,   Tasas     NaN  NaN  NaN  NaN   
3                                                NaN     NaN  NaN  NaN  NaN   
4                                                     Índice  NaN  NaN  NaN   

0  NaN  NaN  NaN  NaN  NaN  
0  NaN  NaN  NaN  NaN  NaN  
1  NaN  NaN  NaN  NaN  NaN  
2  NaN  NaN  NaN  NaN  NaN  
3  NaN  NaN  NaN  NaN  NaN  
4  NaN  NaN  NaN  NaN  NaN  


In [5]:
# Limpieza de datos
print("Iniciando la limpieza de datos...") 

from scripts.limpieza_datos import limpiar_dataframe, reemplazar_nan_por_cero, limpiar_datos_incorrectos

# Limpiar el DataFrame
df_limpio = limpiar_dataframe(df)
archivo_limpio = os.path.join(directorio_salida, 'datos_ine_limpio.xlsx')
df_limpio.to_excel(archivo_limpio, index=False)
print(f"Datos limpiados y guardados en: {archivo_limpio}")

# Reemplazar NaN por 0
df_sin_nan = reemplazar_nan_por_cero(df_limpio)
archivo_sin_nan = os.path.join(directorio_salida, 'datos_ine_sin_nan.xlsx')
df_sin_nan.to_excel(archivo_sin_nan, index=False)
print(f"Datos con NaN reemplazados por 0 y guardados en: {archivo_sin_nan}")

# Limpiar datos incorrectos
df_corregido = limpiar_datos_incorrectos(df_sin_nan)
archivo_corregido = os.path.join(directorio_salida, 'datos_ine_corregido.xlsx')
df_corregido.to_excel(archivo_corregido, index=False)
print(f"Datos incorrectos limpiados y guardados en: {archivo_corregido}")

Iniciando la limpieza de datos...
Datos limpiados y guardados en: data/datos_ine_limpio.xlsx
Datos con NaN reemplazados por 0 y guardados en: data/datos_ine_sin_nan.xlsx
Datos incorrectos limpiados y guardados en: data/datos_ine_corregido.xlsx


In [8]:
# Función para transponer el DataFrame
def transponer_dataframe(df):
    return df.transpose()

# Transponer el DataFrame
df_transpuesto = transponer_dataframe(df_corregido)
archivo_transpuesto = os.path.join(directorio_salida, 'datos_ine_transpuesto.xlsx')
df_transpuesto.to_excel(archivo_transpuesto, index=False)
print(f"Datos transpuestos y guardados en: {archivo_transpuesto}")

# Extraer colecciones de datos
colecciones_datos = [df_transpuesto.iloc[i].tolist() for i in range(len(df_transpuesto))]
print("Colecciones de datos extraídas correctamente.")

Datos transpuestos y guardados en: data/datos_ine_transpuesto.xlsx
Colecciones de datos extraídas correctamente.


In [14]:
import os

# Importar la función guardar_datos_concatenados desde el archivo correspondiente
from scripts.operaciones import guardar_datos_concatenados

# Asegurarse de que el directorio de salida existe
directorio_salida = 'data'
if not os.path.exists(directorio_salida):
    os.makedirs(directorio_salida)

# Guardar las colecciones de datos en un archivo de texto con codificación UTF-8
archivo_lista = os.path.join(directorio_salida, 'lista.txt')
guardar_datos_concatenados(colecciones_datos, archivo_lista)
print(f"Colecciones de datos guardadas en: {archivo_lista}")

NameError: name 'colecciones_datos' is not defined

In [13]:
import os
import pandas as pd
import numpy as np

# Asegurarse de que el directorio de salida existe
directorio_salida = 'data'
if not os.path.exists(directorio_salida):
    os.makedirs(directorio_salida)

# Cargar el DataFrame transpuesto desde el archivo Excel
ruta_archivo_transpuesto = os.path.join(directorio_salida, 'datos_ine_transpuesto.xlsx')
df_transpuesto = pd.read_excel(ruta_archivo_transpuesto)

# Definir las columnas de interés
columnas_interes = [
    'Total industria', 'Bienes de consumo', 'Bienes de consumo duradero',
    'Bienes de consumo no duradero', 'Bienes de equipo', 'Bienes intermedios', 'Energía'
]

# Función para encontrar la fila que contiene las categorías de interés
def encontrar_fila_categorias(df, categorias):
    for idx, row in df.iterrows():
        if all(categoria in ' '.join(row.astype(str)) for categoria in categorias):
            return idx
    return None

# Encontrar la fila que contiene las categorías
fila_categorias = encontrar_fila_categorias(df_transpuesto, columnas_interes)

if fila_categorias is not None:
    # Usar la fila de categorías como encabezados
    df_transpuesto.columns = df_transpuesto.iloc[fila_categorias]
    df_transpuesto = df_transpuesto.iloc[fila_categorias + 1:].reset_index(drop=True)

    # Convertir a numérico
    df_transpuesto = df_transpuesto.apply(pd.to_numeric, errors='coerce')

    # Crear un nuevo DataFrame para almacenar los datos de interés
    df_datos_interes = pd.DataFrame()

    # Recopilar todos los datos para cada categoría de interés
    for categoria in columnas_interes:
        columnas_categoria = [col for col in df_transpuesto.columns if categoria in col]
        datos_categoria = df_transpuesto[columnas_categoria].values.flatten()
        df_datos_interes[categoria] = pd.Series(datos_categoria)

    # Eliminar filas que son completamente NaN
    df_datos_interes = df_datos_interes.dropna(how='all')

    # Guardar el DataFrame con todos los datos en un nuevo archivo Excel
    ruta_archivo_datos = os.path.join(directorio_salida, 'datos_ine_por_categoria.xlsx')
    df_datos_interes.to_excel(ruta_archivo_datos, index=False)
    print(f"DataFrame con todos los datos guardado en: {ruta_archivo_datos}")

    # Imprimir información sobre los datos
    print("\nInformación de los datos por categoría:")
    for columna in columnas_interes:
        datos = df_datos_interes[columna].dropna()
        print(f"{columna}:")
        print(f"  Número de valores: {len(datos)}")
        print(f"  Primeros 5 valores: {datos.head().tolist()}")
        print(f"  Últimos 5 valores: {datos.tail().tolist()}")
        print()

else:
    print("No se encontró una fila que contenga todas las categorías de interés.")

DataFrame con todos los datos guardado en: data/datos_ine_por_categoria.xlsx

Información de los datos por categoría:
Total industria:
  Número de valores: 367740
  Primeros 5 valores: [78.837, 88.496, 81.07, 75.054, 111.844]
  Últimos 5 valores: [0.4, 0.4, 0.4, 0.4, 0.4]

Bienes de consumo:
  Número de valores: 474984
  Primeros 5 valores: [83.304, 74.254, 90.701, 88.873, 119.36]
  Últimos 5 valores: [6.5, 8.5, -11.1, 8.5, 5.0]

Bienes de consumo duradero:
  Número de valores: 367740
  Primeros 5 valores: [68.17, 56.858, 93.734, 68.918, 111.597]
  Últimos 5 valores: [238.6, 238.6, 238.6, 238.6, 238.6]

Bienes de consumo no duradero:
  Número de valores: 367740
  Primeros 5 valores: [84.709, 76.284, 89.638, 89.504, 120.917]
  Últimos 5 valores: [-50.9, -50.9, -50.9, -50.9, -50.9]

Bienes de equipo:
  Número de valores: 367740
  Primeros 5 valores: [65.451, 67.538, 59.153, 79.242, 58.386]
  Últimos 5 valores: [56.7, 56.7, 56.7, 56.7, 56.7]

Bienes intermedios:
  Número de valores: 36774

In [16]:
import os
import pandas as pd
import numpy as np

# Asegurarse de que el directorio de salida existe
directorio_salida = 'data'
if not os.path.exists(directorio_salida):
    os.makedirs(directorio_salida)

# Cargar el DataFrame con los datos por categoría
ruta_archivo_datos = os.path.join(directorio_salida, 'datos_ine_por_categoria.xlsx')
df_datos_interes = pd.read_excel(ruta_archivo_datos)

# Definir las columnas de interés
columnas_interes = [
    'Total industria', 'Bienes de consumo', 'Bienes de consumo duradero',
    'Bienes de consumo no duradero', 'Bienes de equipo', 'Bienes intermedios', 'Energía'
]

# Crear un DataFrame para almacenar las estadísticas
df_estadisticas = pd.DataFrame(columns=['Categoría', 'Media', 'Varianza', 'Moda'])

# Calcular estadísticas para cada categoría de interés
estadisticas = []
for categoria in columnas_interes:
    datos = df_datos_interes[categoria].dropna()
    
    media = datos.mean()
    varianza = datos.var()
    moda = datos.mode().iloc[0] if not datos.empty else None

    estadisticas.append({
        'Categoría': categoria,
        'Media': media,
        'Varianza': varianza,
        'Moda': moda
    })

df_estadisticas = pd.DataFrame(estadisticas)

# Calcular estadísticas globales
media_global = df_estadisticas['Media'].mean()
varianza_global = df_estadisticas['Varianza'].mean()
moda_global = df_estadisticas['Moda'].mode().iloc[0] if not df_estadisticas['Moda'].empty else None

# Guardar las estadísticas en un archivo de texto
ruta_archivo_estadisticas_txt = os.path.join(directorio_salida, 'lista_estadisticas.txt')
with open(ruta_archivo_estadisticas_txt, 'w', encoding='utf-8') as f:
    f.write("Estadísticas por categoría:\n")
    for index, row in df_estadisticas.iterrows():
        f.write(f"{row['Categoría']}: Media={row['Media']}, Varianza={row['Varianza']}, Moda={row['Moda']}\n")
    
    f.write("\nEstadísticas globales:\n")
    f.write(f"MEDIA GLOBAL: {media_global}\n")
    f.write(f"VARIANZA GLOBAL: {varianza_global}\n")
    f.write(f"MODA GLOBAL: {moda_global}\n")

print(f"Estadísticas guardadas en: {ruta_archivo_estadisticas_txt}")

# Imprimir resultados en consola
print("Estadísticas por categoría:")
print(df_estadisticas.to_string(index=False))

print("\nEstadísticas globales:")
print(f"MEDIA GLOBAL: {media_global}")
print(f"VARIANZA GLOBAL: {varianza_global}")
print(f"MODA GLOBAL: {moda_global}")

Estadísticas guardadas en: data/lista_estadisticas.txt
Estadísticas por categoría:
                    Categoría     Media     Varianza  Moda
              Total industria 27.830045  2436.152616   1.3
            Bienes de consumo 89.748157 26681.844156   3.4
   Bienes de consumo duradero 60.074250 35248.948583   0.0
Bienes de consumo no duradero 27.905035  2599.979123   0.7
             Bienes de equipo 32.495436  4150.023898   1.9
           Bienes intermedios 29.430186  3734.279922   1.2
                      Energía 29.631316  3070.528135   3.3

Estadísticas globales:
MEDIA GLOBAL: 42.44491769869906
VARIANZA GLOBAL: 11131.67949042543
MODA GLOBAL: 0.0


In [ ]:
class DataRow:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)
    
    def __str__(self):
        return ', '.join(f"{key}: {value}" for key, value in self.__dict__.items())
    
    def modificar_valor(self, atributo, valor):
        if atributo in self.__dict__:
            self.__dict__[atributo] = valor
    
    def __eq__(self, other):
        return self.__dict__.get('columna2', None) == other.__dict__.get('columna2', None)
    
    def __lt__(self, other):
        return self.__dict__.get('columna2', None) < other.__dict__.get('columna2', None)
    
    def __add__(self, other):
        return {key: self.__dict__[key] + other.__dict__[key] for key in self.__dict__}
    
    def __sub__(self, other):
        return {key: self.__dict__[key] - other.__dict__[key] for key in self.__dict__}

# Crear objetos de la clase para al menos cinco filas
filas = [DataRow(**df_final.iloc[i].to_dict()) for i in range(5)]

# Probar métodos
for fila in filas:
    print(fila)

# Modificar valores
filas[0].modificar_valor('columna1', 999)
print(f"Fila modificada: {filas[0]}")

# Comparar objetos
print(f"Comparación: {filas[0] == filas[1]}")
print(f"Suma: {filas[0] + filas[1]}")
print(f"Resta: {filas[0] - filas[1]}")